#Compfest

In [ ]:
!git clone https://github.com/Billl-11/NewGenAI.git

# Data prep

In [2]:
import pandas as pd
df = pd.read_csv('/content/NewGenAI/Deeplearning NLP/tokenize & padding/tokenizer_padded.csv')
df.head()

,diseases,symptons,tokenizers,padded_tokenizers
0,Fungal infection,"itching, skin rash, nodal skin eruptions, d...",[ 101 1122 7520 117 2241 187 10733 1...,[ 101 1122 7520 117 2241 187 10733 1...
1,Fungal infection,"skin rash, nodal skin eruptions, dischromic...",[ 101 2241 187 10733 117 6873 1348 22...,[ 101 2241 187 10733 117 6873 1348 22...
2,Fungal infection,"nodal skin eruptions, itching, dischromic p...",[ 101 6873 1348 2241 19049 1116 117 11...,[ 101 6873 1348 2241 19049 1116 117 11...
3,Fungal infection,"dischromic patches, skin rash, itching",[ 101 6187 8167 18882 1665 14879 117 22...,[ 101 6187 8167 18882 1665 14879 117 22...
4,Fungal infection,"skin rash, nodal skin eruptions, itching",[ 101 2241 187 10733 117 6873 1348 22...,[ 101 2241 187 10733 117 6873 1348 22...


## One hot label

In [3]:
one_hot_df = pd.get_dummies(df['diseases'])
# Get the unique class labels from the DataFrame columns
class_labels = one_hot_df.columns.tolist()

# Create a label mapping from class labels to indices
label_mapping = {i: label for i, label in enumerate(class_labels)}
inv_map = {v: k for k, v in label_mapping.items()}
print("Label Mapping:")
print(inv_map)

Label Mapping:
{'(vertigo) Paroymsal  Positional Vertigo': 0, 'AIDS': 1, 'Acne': 2, 'Alcoholic hepatitis': 3, 'Allergy': 4, 'Arthritis': 5, 'Bronchial Asthma': 6, 'Cervical spondylosis': 7, 'Chicken pox': 8, 'Chronic cholestasis': 9, 'Common Cold': 10, 'Dengue': 11, 'Diabetes ': 12, 'Dimorphic hemmorhoids(piles)': 13, 'Drug Reaction': 14, 'Fungal infection': 15, 'GERD': 16, 'Gastroenteritis': 17, 'Heart attack': 18, 'Hepatitis B': 19, 'Hepatitis C': 20, 'Hepatitis D': 21, 'Hepatitis E': 22, 'Hypertension ': 23, 'Hyperthyroidism': 24, 'Hypoglycemia': 25, 'Hypothyroidism': 26, 'Impetigo': 27, 'Jaundice': 28, 'Malaria': 29, 'Migraine': 30, 'Osteoarthristis': 31, 'Paralysis (brain hemorrhage)': 32, 'Peptic ulcer diseae': 33, 'Pneumonia': 34, 'Psoriasis': 35, 'Tuberculosis': 36, 'Typhoid': 37, 'Urinary tract infection': 38, 'Varicose veins': 39, 'hepatitis A': 40}


In [41]:
num_class = len(inv_map)
print("Number of label:",num_class)

Number of label: 41


In [4]:
df["diseases"] = df.diseases.map(inv_map)

In [5]:
df.head()

,diseases,symptons,tokenizers,padded_tokenizers
0,15,"itching, skin rash, nodal skin eruptions, d...",[ 101 1122 7520 117 2241 187 10733 1...,[ 101 1122 7520 117 2241 187 10733 1...
1,15,"skin rash, nodal skin eruptions, dischromic...",[ 101 2241 187 10733 117 6873 1348 22...,[ 101 2241 187 10733 117 6873 1348 22...
2,15,"nodal skin eruptions, itching, dischromic p...",[ 101 6873 1348 2241 19049 1116 117 11...,[ 101 6873 1348 2241 19049 1116 117 11...
3,15,"dischromic patches, skin rash, itching",[ 101 6187 8167 18882 1665 14879 117 22...,[ 101 6187 8167 18882 1665 14879 117 22...
4,15,"skin rash, nodal skin eruptions, itching",[ 101 2241 187 10733 117 6873 1348 22...,[ 101 2241 187 10733 117 6873 1348 22...


In [6]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(df.diseases)

In [ ]:
labels[0]

# Train - Test

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.symptons, labels, test_size=0.20, stratify = labels, random_state=42)

In [ ]:
y_train[0]

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer,TFBertModel, AutoModel
# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
# bert = TFBertModel.from_pretrained('bert-base-cased')
model_b = TFBertModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
Tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",do_lower_case=True)

In [12]:
maxlen = 50

In [14]:

x_train = Tokenizer(
    text=X_train.tolist(),
    add_special_tokens=True,
    max_length=maxlen,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = Tokenizer(
    text=X_test.tolist(),
    add_special_tokens=True,
    max_length=maxlen,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [38]:
maxlen_train = len(x_train[0])

In [15]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

# Dense Layer

In [16]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense,LSTM,Flatten

In [78]:
import tensorflow
input_ids = Input(shape=(maxlen_train,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(maxlen_train,), dtype=tf.int32, name="attention_mask")

# bert base model
# embeddings = bert(input_ids,attention_mask = input_mask)[0]

# bert clinical model
embeddings = model_b(input_ids,attention_mask = input_mask)[0]
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)

# LSTM
# lstm = LSTM(units=64, dropout=0.3, recurrent_dropout=0.3)(embeddings)

# out = Flatten()(lstm)

# out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.3)(out)
out = Dense(64,activation = 'relu')(out)
y = Dense(num_class,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
model.summary()

In [80]:
from tensorflow.keras.optimizers.legacy import Adam

optimizer = Adam(
    learning_rate=0.000001, # this learning rate is for bert model , taken from huggingface website
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = metric)

In [81]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=1,
    batch_size=32
)

/usr/local/lib/python3.10/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


306/306 [==============================] - 119s 299ms/step - loss: 3.6480 - balanced_accuracy: 0.0722 - val_loss: 3.1101 - val_balanced_accuracy: 0.2486


# Model Performance

In [72]:
model.save('/content/model_clinicalbert-ver2.h5')

In [73]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import transformers
# model = tf.keras.models.load_model('/content/drive/MyDrive/model_ver_test.h5', custom_objects={"TFBertModel": transformers.TFBertModel})
model = tf.keras.models.load_model('/content/model_clinicalbert.h5', custom_objects={"TFBertModel": transformers.TFBertModel})

In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

In [75]:
import numpy as np

# Initialize an empty list to store the decoded numerical labels
decoded_numerical_labels = []
for one_hot_label in y_test:
    label_index = np.argmax(one_hot_label)
    numerical_label = label_index
    decoded_numerical_labels.append(numerical_label)

y_test_cls = np.array(decoded_numerical_labels)

In [76]:
import numpy as np
y_predicted = np.argmax(predicted_raw, axis = 1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_cls, y_predicted))

# Test model

In [ ]:
texts = 'itching, skin rash, nodal skin eruptions'
x_val = Tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=36,
    truncation=True,
    padding='max_length',
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(inv_map.keys(),validation[0]):
    print(key,value)

In [86]:
def test():
  # Step 1: Initialize an empty dictionary
  result_dict = {}

  # Step 2: Populate the dictionary
  for key, value in zip(inv_map.keys(), validation[0]):
      result_dict[key] = value

  # Step 3: Sort the dictionary by values in descending order
  sorted_dict = dict(sorted(result_dict.items(), key=lambda item: item[1], reverse=True))

  # Step 4: Retrieve the top 3 key-value pairs
  top_3 = list(sorted_dict.items())[:3]
  top3_class_labels = []
  top3_probabilities = []
  # Print the top 3 key-value pairs
  for key, value in top_3:
      top3_class_labels.append(key)
      top3_probabilities.append(value)
  return top3_class_labels,top3_probabilities

In [ ]:
test()